# Nuestra primer aplicacion

## Atencion
Para ejecutar esta libreta primero tienen que configurar su sistema como se describe en [la libreta de preparacion](./L00_Preparacion.ipynb). Comenzaremos por conectarnos al dispositivo y asegurar que podemos comunicarnos con el.

In [65]:
!adb devices
!adb forward tcp:9999 tcp:2222

In [66]:
import android

In [67]:
droid = android.Android()
droid.ttsSpeak("listo para recibir instrucciones|")

Result(id=0, result=None, error=None)

## Objetivo
En la sesion de hoy tendremos como objetivo escribir una aplicacion que se conecte a un servicio externo y obtenga la informacion del estado del tiempo en la posicion actual del dispositivo, y finalmente que el dispositivo "hable" el estado del tiempo descargado. Para esto necesitaremos:

1. conocer la ubicacion del dispositivo
2. solcitar la informacion del estado del tiempo en la ubicacion del dispositivo

Para 1. usaremos la fachada de localizacion , en particular usaremos [getLastKnownLocation](https://github.com/kuri65536/sl4a/blob/master/docs/ApiReference.md#getlastknownlocation) y [geocode](https://github.com/kuri65536/sl4a/blob/master/docs/ApiReference.md#geocode).
Para 2. usaremos la informacion obtenida de 1. y algunas librerias externas de Python para solicitar el estado del tiempo usando la API de [OpenWeatherMap](https://openweathermap.org/city/3526617).

Comenzamos!

## 1. Determinar ultima posicion conocida del dispositivo

In [68]:
# obtener la ultima localizacion conocida
droid.getLastKnownLocation().result

{u'gps': None,
 u'network': {u'accuracy': 28.100000381469727,
  u'altitude': 1346,
  u'bearing': 0,
  u'latitude': 19.519619,
  u'longitude': -96.9155692,
  u'provider': u'network',
  u'speed': 0,
  u'time': 1521559881141},
 u'passive': {u'accuracy': 28.100000381469727,
  u'altitude': 1346,
  u'bearing': 0,
  u'latitude': 19.519619,
  u'longitude': -96.9155692,
  u'provider': u'network',
  u'speed': 0,
  u'time': 1521559881141}}

In [69]:
# asignamos la ultima localizacion conocida a una variable.
# notemos que la variable es del tipo diccionario (dict)
ultimaLocalizacionConocida = droid.getLastKnownLocation().result

type(ultimaLocalizacionConocida)

dict

In [11]:
# notemos que el valor de la llave 'gps' es nulo
print(ultimaLocalizacionConocida['gps'])

None


In [14]:
# accedemos al valor de la llave network
ultimaLocalizacionConocida['network']

{u'accuracy': 30,
 u'altitude': 1346,
 u'bearing': 0,
 u'latitude': 19.519619,
 u'longitude': -96.9155692,
 u'provider': u'network',
 u'speed': 0,
 u'time': 1521486572821}

El valor asignado a la variable `ultimaLoc` dependera de cual de los proveedores sea no nulo. Al final de la celda desplegamos el contenido asignado a `ultimaLoc`. Noten la estructura de diccionario.

In [20]:
# asignamos una variable al contenido de la llave que no es nula usando una condicional

if ultimaLocalizacionConocida['gps'] is not None:
    ultimaLoc = ultimaLocalizacionConocida['gps']
else:
    ultimaLoc = ultimaLocalizacionConocida['network']

print type(ultimaLoc)
print ultimaLoc
ultimaLoc.keys()

<type 'dict'>
{u'bearing': 0, u'altitude': 1346, u'time': 1521486572821, u'longitude': -96.9155692, u'provider': u'network', u'latitude': 19.519619, u'speed': 0, u'accuracy': 30}


[u'bearing',
 u'altitude',
 u'time',
 u'longitude',
 u'provider',
 u'latitude',
 u'speed',
 u'accuracy']

Ahora vamos a extraer la latitud, longitud y estampa temporal del diccionario anterior. 

In [37]:
# del diccionario extraemos latitud y longitud y fecha
lat = ultimaLoc["latitude"]
lon = ultimaLoc["longitude"]
time = ultimaLoc["time"]
print time

1521486572821


Noten que la variable `time` es lo que se conoce como un `timestamp` un numero entero que cuenta los segundos transcurridos desde un inicio determinado. De este modo necesitamos convertirla en un formato amigable. Para esto usamos un modulo de Python que se llama `datetime`

In [39]:
import datetime

fecha = datetime.datetime.fromtimestamp(time / 1e3)
print fecha

2018-03-19 13:09:32.821000


In [43]:
# otra forma de presentar la fecha solo hasta segundos
fecha = datetime.datetime.fromtimestamp(time/1e3).strftime('%Y-%m-%d %H:%M:%S')
print fecha

2018-03-19 13:09:32


Ahora usaremos [geocode](https://github.com/kuri65536/sl4a/blob/master/docs/ApiReference.md#geocode) que es una funcion del dispositivo que regresa una lista de direcciones para la latitud y longitud dadas:

In [29]:
droid.geocode(lat,lon,5).result

[{u'admin_area': u'Veracruz',
  u'country_code': u'MX',
  u'country_name': u'Mexico',
  u'feature_name': u'68',
  u'locality': u'Xalapa Enr\xedquez',
  u'postal_code': u'91000',
  u'sub_admin_area': u'Xalapa',
  u'thoroughfare': u'Avenida Enrique C. R\xe9bsamen'},
 {u'admin_area': u'Veracruz',
  u'country_code': u'MX',
  u'country_name': u'Mexico',
  u'feature_name': u'Centro',
  u'locality': u'Xalapa Enr\xedquez',
  u'postal_code': u'91000',
  u'sub_admin_area': u'Xalapa'},
 {u'admin_area': u'Veracruz',
  u'country_code': u'MX',
  u'country_name': u'Mexico',
  u'feature_name': u'Zona Centro',
  u'locality': u'Xalapa Enr\xedquez',
  u'postal_code': u'91000',
  u'sub_admin_area': u'Xalapa'},
 {u'admin_area': u'Veracruz',
  u'country_code': u'MX',
  u'country_name': u'Mexico',
  u'feature_name': u'Xalapa',
  u'locality': u'Xalapa',
  u'sub_admin_area': u'Xalapa'},
 {u'admin_area': u'Veracruz',
  u'country_code': u'MX',
  u'country_name': u'Mexico',
  u'feature_name': u'91000',
  u'locali

Noten que el resultado no es un diccionario, sino una lista de diccionarios. Si no se especifica el ultimo numero (cuantas direcciones solicitar) se regresa solo un valor:

In [31]:
direccion = droid.geocode(lat,lon).result
print direccion
type(direccion)

[{u'thoroughfare': u'Avenida Enrique C. R\xe9bsamen', u'locality': u'Xalapa Enr\xedquez', u'sub_admin_area': u'Xalapa', u'admin_area': u'Veracruz', u'feature_name': u'68', u'country_code': u'MX', u'country_name': u'Mexico', u'postal_code': u'91000'}]


list

Dado que la variable `direccion` es una lista, debemos acceder a sus elementos especificando un indice:

In [33]:
codigoPostal = direccion[0]["postal_code"]
print codigoPostal
codigoPais = direccion[0]["country_code"]
print codigoPais

91000
MX


## 2. Solicitar estado del tiempo usando API de OpenWeatherMap

Con el codigo postal y el codigo del pais podemos solicitar informacion a la API en la siguiente url:

In [44]:
url="http://api.openweathermap.org/data/2.5/weather?zip={0},{1}&APPID=dfeea5e604e1948b28652216ad910d11".format(codigoPostal,codigoPais)

In [45]:
print(url)

http://api.openweathermap.org/data/2.5/weather?zip=91000,MX&APPID=dfeea5e604e1948b28652216ad910d11


Podemos solicitar la informacion a un `url` directamente usando una libreria que se llama `requests` del siguiente modo:

In [114]:
import requests
estadoDelTiempo = requests.get(url).json()

# vamos a usar urrlib2 pues requests no esta disponible en el movil

import urllib2
response = urllib2.urlopen(url)
respuesta = response.read()
response.close()

In [115]:
# convertimos el resultado a un dictionario de JSON

import json

estadoDelTiempo = json.loads(respuesta)

El resultado es un diccionario en formato json

In [116]:
estadoDelTiempo

{u'base': u'stations',
 u'clouds': {u'all': 5},
 u'cod': 200,
 u'coord': {u'lat': 19.43, u'lon': -99.13},
 u'dt': 1521568800,
 u'id': 590134368,
 u'main': {u'humidity': 17,
  u'pressure': 1024,
  u'temp': 298.56,
  u'temp_max': 303.15,
  u'temp_min': 295.15},
 u'name': u'Mexico City',
 u'sys': {u'country': u'MX',
  u'id': 3998,
  u'message': 0.0038,
  u'sunrise': 1521549591,
  u'sunset': 1521593265,
  u'type': 1},
 u'visibility': 11265,
 u'weather': [{u'description': u'clear sky',
   u'icon': u'02d',
   u'id': 800,
   u'main': u'Clear'}],
 u'wind': {u'deg': 40, u'speed': 2.1}}

In [109]:
estadoDelTiempo.keys()

[u'clouds',
 u'name',
 u'visibility',
 u'sys',
 u'weather',
 u'coord',
 u'base',
 u'dt',
 u'main',
 u'id',
 u'wind',
 u'cod']

In [110]:
# queremos la descripcion de estado del tiempo
estadoDelTiempo["weather"]

[{u'description': u'clear sky',
  u'icon': u'02d',
  u'id': 800,
  u'main': u'Clear'}]

In [52]:
# es una lista
descripcion = estadoDelTiempo["weather"][0]['description']

In [111]:
import sys
reload(sys)  # Reload does the trick!
sys.setdefaultencoding('UTF8')

# Ejercicio

Completar los valores que hacen falta entre las comas al final de la siguiente instruccion.
Necesitan ingresar cuatro valores entre las comas, los valores estan en alguna de las celdas de evaluacion de esta libreta.
El mensaje debe verse como sigue:

    'En la ciudad de Xalapa Enr\xc3\xadquez, siendo las 2018-03-19 13:09:32 horas, en la calle de Avenida Enrique C. R\xc3\xa9bsamen, el estado del tiempo es scattered clouds'

In [61]:
# ejercicio, completa los valores que hacen falta entre las comas
mensaje='En la ciudad de {0}, siendo las {1} horas, en la calle de {2}, el estado del tiempo es {3}'.format( \
                                ,,,)

In [112]:
mensaje

'En la ciudad de Xalapa Enr\xc3\xadquez, siendo las 2018-03-19 13:09:32 horas, en la calle de Avenida Enrique C. R\xc3\xa9bsamen, el estado del tiempo es scattered clouds'

Finalmente le damos al dispositivo la instruccion de darnos el mensaje creado...

In [113]:
droid.ttsSpeak(mensaje)

ValueError: No JSON object could be decoded

Felicidades! han hecho su primer app en Python para Android. En la siguiente sesion vamos a integrar la app con la funcionalidad de localizacion para que pueda decirles el estado del tiempo en cualquier lugar en donde se encuentren usando la posicion actual.